In [1]:
using Revise

using Kate

### Check Environment

In [2]:
Kate.test()

Testing environment...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


### Set Parameters

In [5]:
# Sample name
sa = "human_dna_low_quality"

# Molecule: cdna or dna
mo = "dna"

# Adapter sequence to trim
ad = "AGATCGGAAGAGC"

# Forward read file
fq1 = "../input/human_dna_low_quality/SRR957824_500K_R1.fastq.gz"

# Reverse read file
fq2 = "../input/human_dna_low_quality/SRR957824_500K_R2.fastq.gz"

# is dna targeted
ta = false

# Number of jobs
n_jo = 6

# G of memory
met = 40

# G of memory per job
mej = 2;

In [6]:
pap = dirname(@__DIR__)

pao = joinpath(pap, "output")

pai = joinpath(pap, "input")

par = joinpath(pai, "human_reference_genome")

"/home/jovyan/Kate.jl/input/human_reference_genome"

### Process DNA

In [7]:
Kate.process_dna(
    mo,
    fq1,
    fq2,
    ad,
    ta,
    joinpath(pao, string(sa)),
    joinpath(par, "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz"),
    joinpath(par, "chromosome.bed.gz"),
    joinpath(par, "chrn_n.tsv"),
    "/opt/snpeff/snpEff/snpEff.jar",
    n_jo,
    met,
    mej,
)

Skipping trimming because trimmed files already exist:
 /home/jovyan/Kate.jl/output/human_dna_low_quality/trim/trimmed-pair1.fastq.gz
 /home/jovyan/Kate.jl/output/human_dna_low_quality/trim/trimmed-pair2.fastq.gz

Skipping check sequence because directory already exists:
 /home/jovyan/Kate.jl/output/human_dna_low_quality/check_trim

Done at 2021-08-20T20:34:41.431 in empty period.

Skipping alignment because directory already exists: 
/home/jovyan/Kate.jl/output/human_dna_low_quality/align
Done at 2021-08-20T20:34:41.451 in empty period.

`bash -c 'source activate py2 && configManta.py --referenceFasta /home/jovyan/Kate.jl/input/human_reference_genome/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bgz --callRegions /home/jovyan/Kate.jl/input/human_reference_genome/chromosome.bed.gz --bam /home/jovyan/Kate.jl/output/human_dna_low_quality/align/germ.bam --outputContig --runDir /home/jovyan/Kate.jl/output/human_dna_low_quality/call_variant/manta && /home/jovyan/Kate.jl/output/human_dna_l

LoadError: [91mfailed process: Process(`[4mbash[24m [4m-c[24m [4m'source activate py2 && configManta.py --referenceFasta /home/jovyan/Kate.jl/input/human_reference_genome/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bgz --callRegions /home/jovyan/Kate.jl/input/human_reference_genome/chromosome.bed.gz --bam /home/jovyan/Kate.jl/output/human_dna_low_quality/align/germ.bam --outputContig --runDir /home/jovyan/Kate.jl/output/human_dna_low_quality/call_variant/manta && /home/jovyan/Kate.jl/output/human_dna_low_quality/call_variant/manta/runWorkflow.py --mode local --jobs 6 --memGb 40 --quiet'[24m`, ProcessExited(1)) [1][39m


## Benchmark

TODO: Refactor

In [ ]:
# baseline vcf
vb = joinpath(pai, "high_confidence_HG002_4.1_GRCh38/HG002_GRCh38_1_22_v4.1_draft_benchmark.vcf.gz")

# calls vcf
vc = joinpath(pao, "HG002_cat/find_variant/strelka/results/variants/variants.vcf.gz")

sdf = joinpath(pai, "GRCh38.sdf")

pab = joinpath(pao, "HG002_cat/benchmark")

rtg = "/opt/rtg/rtg-tools-3.11/rtg"

### Configure vcfeval

In [ ]:
run(`$rtg`)

### Download Real Time Genomics prepard GRCH38 files
vcfeval requires reference files in RTG formats. RTG makes versions of these files for the popular reference genomes such as GRCh38, which will be downloaded below.

In [ ]:
if ispath(sdf)
    
    println("Skipping download because RTG GRCh38 data directory already exists:\n $sdf")

else

    run(pipeline(
            `wget -q -P ../input/ https://s3.amazonaws.com/rtg-datasets/references/GRCh38.sdf.zip`,
            `unzip -d ../input/ ../input/GRCh38.sdf.zip`,
            `rm ../input/GRCh38.sdf.zip`,
            ))
end

### Run vcfeval
This command compares the VCF you generated (calls) to the highly confident VCF generated by Genome in a Bottle (baseline).

In [ ]:
run(`$rtg vcfeval
    --baseline=$vb
    --calls=$vc
    --template=$sdf
    --output=$pab
    --threads=1
    `)

### See results
Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

In [ ]:
# vcfeval summary file
pasu = open(joinpath(pab, "summary.txt"))

su = read(pasu)

su